In [6]:
from preprocess.hand_detection import HandDetector
from recognizer.simple_classifier import HandGestureClassifier
from capture.camera import get_camera_stream
import cv2
from IPython.display import display, clear_output
import time

print("""
INSTRUCCIONES para capturar las señas:

- hello: Mano saludando, como decir "hello" 👋
- yes: Puño que asiente afirmando
- love: Forma "I love you" (dedos índice, meñique y pulgar)

Recuerda hacer cada seña de forma clara y pausada, manteniéndola unos segundos.

Presiona Enter para comenzar a capturar cada seña.
""")

detector = HandDetector()
classifier = HandGestureClassifier()

labels = ["hello", "yes", "love"]
X, y = [], []

cap = get_camera_stream()

for label in labels:
    input(f"\nPresiona Enter para capturar 30 frames de '{label}'...")
    count = 0
    while count < 30:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer frame de la cámara.")
            break
        
        results = detector.detect(frame)
        detector.draw_landmarks(frame, results)

        if results.multi_hand_landmarks:
            emb = classifier.extract_landmarks(results.multi_hand_landmarks[0])
            if emb is not None:
                X.append(emb)
                y.append(label)
                count += 1
                print(f"{label}: {count}/30", end='\r')

        # Agregar texto al frame
        cv2.putText(frame, f"Capturando: {label} ({count}/30)", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        # Convertir imagen BGR a RGB para mostrarla con IPython.display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Mostrar imagen en el notebook
        clear_output(wait=True)
        display(frame_rgb)

        # Pequeña pausa para que se actualice la imagen
        time.sleep(0.05)

cap.release()

if len(X) > 0:
    classifier.fit(X, y)
    classifier.save("models/simple_svm_model.pkl")
    print("\n✅ Modelo entrenado y guardado en models/simple_svm_model.pkl")
else:
    print("\nNo se capturaron datos, modelo no entrenado.")

array([[[146, 152, 141],
        [146, 152, 141],
        [143, 151, 144],
        ...,
        [114, 113, 120],
        [113, 113, 113],
        [114, 114, 114]],

       [[144, 153, 143],
        [143, 151, 142],
        [142, 150, 141],
        ...,
        [122, 122, 124],
        [116, 116, 116],
        [114, 114, 114]],

       [[143, 151, 142],
        [145, 154, 144],
        [154, 155, 148],
        ...,
        [122, 122, 122],
        [125, 126, 118],
        [126, 127, 120]],

       ...,

       [[165, 162, 163],
        [165, 162, 163],
        [159, 159, 159],
        ...,
        [141, 141, 141],
        [142, 147, 145],
        [142, 147, 145]],

       [[163, 163, 163],
        [158, 158, 158],
        [155, 155, 155],
        ...,
        [140, 140, 140],
        [144, 144, 144],
        [145, 145, 145]],

       [[155, 155, 155],
        [156, 156, 156],
        [157, 157, 157],
        ...,
        [140, 140, 140],
        [144, 144, 144],
        [144, 144, 144]]


✅ Modelo entrenado y guardado en models/simple_svm_model.pkl
